In [3]:
import pandas as pd 
import numpy as np
import requests
import datetime

In [5]:
def stateNames(stateAbbreviation):
    states = {
            'AK': 'Alaska',
            'AL': 'Alabama',
            'AR': 'Arkansas',
            'AS': 'American Samoa',
            'AZ': 'Arizona',
            'CA': 'California',
            'CO': 'Colorado',
            'CT': 'Connecticut',
            'DC': 'District of Columbia',
            'DE': 'Delaware',
            'FL': 'Florida',
            'GA': 'Georgia',
            'GU': 'Guam',
            'HI': 'Hawaii',
            'IA': 'Iowa',
            'ID': 'Idaho',
            'IL': 'Illinois',
            'IN': 'Indiana',
            'KS': 'Kansas',
            'KY': 'Kentucky',
            'LA': 'Louisiana',
            'MA': 'Massachusetts',
            'MD': 'Maryland',
            'ME': 'Maine',
            'MI': 'Michigan',
            'MN': 'Minnesota',
            'MO': 'Missouri',
            'MP': 'Northern Mariana Islands',
            'MS': 'Mississippi',
            'MT': 'Montana',
            'NA': 'National',
            'NC': 'North Carolina',
            'ND': 'North Dakota',
            'NE': 'Nebraska',
            'NH': 'New Hampshire',
            'NJ': 'New Jersey',
            'NM': 'New Mexico',
            'NV': 'Nevada',
            'NY': 'New York',
            'OH': 'Ohio',
            'OK': 'Oklahoma',
            'OR': 'Oregon',
            'PA': 'Pennsylvania',
            'PR': 'Puerto Rico',
            'RI': 'Rhode Island',
            'SC': 'South Carolina',
            'SD': 'South Dakota',
            'TN': 'Tennessee',
            'TX': 'Texas',
            'UT': 'Utah',
            'VA': 'Virginia',
            'VI': 'Virgin Islands',
            'VT': 'Vermont',
            'WA': 'Washington',
            'WI': 'Wisconsin',
            'WV': 'West Virginia',
            'WY': 'Wyoming'
    }
    if stateAbbreviation is not None:
        if stateAbbreviation in states:
            return states[stateAbbreviation]
        else:
            return None
    else:
        return None

In [3]:
#Veteran Population Projection Estimates by County (Estimates span several decades; 2021 is selected out below)
veterans = pd.read_csv('https://raw.githubusercontent.com/trebbag/VACovid/main/VetPop2018_County_Data__9L.csv',dtype={'FIPS':'str'})

veterans['Date'] = pd.to_datetime(veterans['Date'])
veterans2021 = veterans[veterans['Date'] == '2021-09-30'].reset_index(drop=True)
veterans2021['County'] = veterans2021['County, St'].str.split(',').str[0]

#Remove unneeded columns as well as Gender and then sum to remove Gender split of data
veterans2021 = veterans2021.drop(['County, St', 'Date', 'Gender'],axis=1)
veterans2021 = (veterans2021.groupby(['FIPS','Age Group','County'], sort=False, as_index=False)
                    .agg({'Veterans':'sum', 'State':'first'})
                    .reindex(columns=veterans2021.columns)) 

#Remove non-CONUS areas, reorder columns, and change column names for consistency
veterans2021.drop(veterans2021[veterans2021['State'] == 'Island Areas & Foreign'].index, inplace=True)
veterans2021.drop(veterans2021[veterans2021['State'] == 'Puerto Rico'].index, inplace=True)
veterans2021.loc[(veterans2021.FIPS == '11001'),'State'] = 'District of Columbia'
veterans2021 = veterans2021[['FIPS','County','State','Age Group','Veterans']]
veterans2021 = veterans2021.rename(columns={'County':'COUNTY','Age Group':'AGEGROUP','State':'STATE','Veterans':'VETS'})

with pd.option_context('display.expand_frame_repr', False): 
    print(veterans2021)

        FIPS    COUNTY    STATE AGEGROUP         VETS
0      01001   Autauga  Alabama    17-44  1355.750242
1      01001   Autauga  Alabama    45-64  2237.755474
2      01001   Autauga  Alabama    65-84  1821.970552
3      01001   Autauga  Alabama      85+   287.713197
4      01003   Baldwin  Alabama    17-44  2936.527243
...      ...       ...      ...      ...          ...
12563  56043  Washakie  Wyoming      85+    37.780506
12564  56045    Weston  Wyoming    17-44   112.840273
12565  56045    Weston  Wyoming    45-64   160.579994
12566  56045    Weston  Wyoming    65-84   263.074304
12567  56045    Weston  Wyoming      85+    12.554431

[12568 rows x 5 columns]


In [4]:
#Create a dataframe without age column to get total counts of veterans by county
POPveterans2021 = veterans2021.drop(['AGEGROUP'],axis=1)

POPveterans2021 = (POPveterans2021.groupby(['FIPS','COUNTY'], sort=False, as_index=False)
                    .agg({'VETS':'sum', 'STATE':'first'})
                    .reindex(columns=POPveterans2021.columns))

with pd.option_context('display.expand_frame_repr', False): 
    print(POPveterans2021)

       FIPS      COUNTY    STATE          VETS
0     01001     Autauga  Alabama   5703.189466
1     01003     Baldwin  Alabama  19665.803702
2     01005     Barbour  Alabama   1848.845078
3     01007        Bibb  Alabama   1264.859815
4     01009      Blount  Alabama   3703.698806
...     ...         ...      ...           ...
3137  56037  Sweetwater  Wyoming   3161.770380
3138  56039       Teton  Wyoming   1093.840038
3139  56041       Uinta  Wyoming   1218.901127
3140  56043    Washakie  Wyoming    621.987939
3141  56045      Weston  Wyoming    549.049003

[3142 rows x 4 columns]


In [5]:
API_KEY = 'eed39902208fc176e948f1dc4c8ecd60a81fd8d1'
AGEPOP_API_URL = 'https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP,AGEGROUP&for=county:*&in=state:*&key={}'.format(API_KEY)
POP_API_URL = 'https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&for=county:*&in=state:*&key={}'.format(API_KEY)

#API Call for Census data by age groups 
results = requests.get(AGEPOP_API_URL).json()
agepop = pd.DataFrame(results[1:], columns=results[0])

#Cleaning of Population by Age Group data
agepop['FIPS'] = agepop.state + agepop.county
agepop = agepop.astype(dtype={'NAME': 'str', 'POP':'float','AGEGROUP':'int64','state':'str','county':'str'})
agepop = agepop.sort_values(by=['NAME','AGEGROUP'])
agepop[['COUNTY','STATE']] = agepop['NAME'].str.split(', ', expand=True)
agepop['COUNTY'] = agepop['COUNTY'].str.replace(' County', '')
agepop['COUNTY'] = agepop['COUNTY'].str.replace(' Parish', '')
agepop = agepop[['FIPS','COUNTY','STATE','AGEGROUP','POP']].reset_index(drop=True)
agepop.drop(agepop[agepop['STATE'].str.contains('Puerto Rico')].index, inplace=True)
agepop = agepop.sort_values(by=['FIPS','AGEGROUP']).reset_index(drop=True)
agepop.head()

,FIPS,COUNTY,STATE,AGEGROUP,POP
0,01001,Autauga,Alabama,0,55869.0
1,01001,Autauga,Alabama,1,3277.0
2,01001,Autauga,Alabama,2,3465.0
3,01001,Autauga,Alabama,3,3851.0
4,01001,Autauga,Alabama,4,3659.0


In [29]:
AG1 = agepop[agepop['AGEGROUP'] == 30].rename(columns={'POP':'TOTPOP1'}).drop(['AGEGROUP'], axis=1).reset_index(drop=True)
AG2 = agepop[agepop['AGEGROUP'] == 25].rename(columns={'POP':'TOTPOP2'}).drop(['AGEGROUP'], axis=1).reset_index(drop=True)

AG3 = agepop[agepop['AGEGROUP'].isin([14,15,16,17])].rename(columns={'POP':'TOTPOP3'}).drop(['AGEGROUP'], axis=1).reset_index(drop=True)
AG3 = (AG3.groupby(['FIPS','COUNTY','STATE'], sort=False, as_index=False).agg({'TOTPOP3':'sum'}))

AG4 = agepop[agepop['AGEGROUP'] == 18].rename(columns={'POP':'TOTPOP4'}).drop(['AGEGROUP'], axis=1).reset_index(drop=True)

AGKEY = ['FIPS','STATE']

AG = AG1.merge(AG2,on=AGKEY,how='left',suffixes=('','_drop')).merge(AG3,on=AGKEY,how='left',suffixes=('','_drop')).merge(AG4,on=AGKEY,how='left',suffixes=('','_drop'))
AG.drop([col for col in AG.columns if 'drop' in col],axis=1,inplace=True)
print(AG)

       FIPS      COUNTY    STATE  TOTPOP1  TOTPOP2  TOTPOP3  TOTPOP4
0     01001     Autauga  Alabama  21434.0  14918.0   7988.0    936.0
1     01003     Baldwin  Alabama  76014.0  61206.0  42384.0   4446.0
2     01005     Barbour  Alabama   9301.0   6331.0   4392.0    469.0
3     01007        Bibb  Alabama   8890.0   6058.0   3344.0    389.0
4     01009      Blount  Alabama  20721.0  15356.0   9714.0   1100.0
...     ...         ...      ...      ...      ...      ...      ...
3137  56037  Sweetwater  Wyoming  17371.0  10337.0   5026.0    463.0
3138  56039       Teton  Wyoming  10033.0   6184.0   3360.0    357.0
3139  56041       Uinta  Wyoming   7516.0   4946.0   2747.0    278.0
3140  56043    Washakie  Wyoming   2628.0   2049.0   1486.0    244.0
3141  56045      Weston  Wyoming   2363.0   1928.0   1278.0    200.0

[3142 rows x 7 columns]


In [31]:
AGVet1 = veterans2021[veterans2021['AGEGROUP'] == '17-44'].rename(columns={'VETS':'VETPOP1'}).drop(['AGEGROUP'],axis=1).reset_index(drop=True)
AGVet2 = veterans2021[veterans2021['AGEGROUP'] == '45-64'].rename(columns={'VETS':'VETPOP2'}).drop(['AGEGROUP'],axis=1).reset_index(drop=True)
AGVet3 = veterans2021[veterans2021['AGEGROUP'] == '65-84'].rename(columns={'VETS':'VETPOP3'}).drop(['AGEGROUP'],axis=1).reset_index(drop=True)
AGVet4 = veterans2021[veterans2021['AGEGROUP'] == '85+'].rename(columns={'VETS':'VETPOP4'}).drop(['AGEGROUP'],axis=1).reset_index(drop=True)

AGVet = AGVet1.merge(AGVet2,on=AGKEY,how='left',suffixes=('','_drop')).merge(AGVet3,on=AGKEY,how='left',suffixes=('','_drop')).merge(AGVet4,on=AGKEY,how='left',suffixes=('','_drop'))
AGVet.drop([col for col in AGVet.columns if 'drop' in col],axis=1,inplace=True)
print(AGVet)

       FIPS      COUNTY    STATE      VETPOP1      VETPOP2      VETPOP3  \
0     01001     Autauga  Alabama  1355.750242  2237.755474  1821.970552   
1     01003     Baldwin  Alabama  2936.527243  5989.212268  9060.702094   
2     01005     Barbour  Alabama   318.631790   706.117241   727.108396   
3     01007        Bibb  Alabama   194.298055   437.829733   559.635620   
4     01009      Blount  Alabama   667.709857  1097.724987  1640.854655   
...     ...         ...      ...          ...          ...          ...   
3137  56037  Sweetwater  Wyoming   757.539729   963.783489  1317.010899   
3138  56039       Teton  Wyoming   119.684117   322.388664   607.847650   
3139  56041       Uinta  Wyoming   220.408779   377.312506   577.140590   
3140  56043    Washakie  Wyoming   107.418894   158.755930   318.032608   
3141  56045      Weston  Wyoming   112.840273   160.579994   263.074304   

          VETPOP4  
0      287.713197  
1     1679.362098  
2       96.987650  
3       73.096405  

In [8]:
#API Call for total population counts not split by age groups
results2 = requests.get(POP_API_URL).json()
pop = pd.DataFrame(results2[1:], columns=results2[0])

#Cleaning of total population counts
pop['FIPS'] = pop.state + pop.county
pop = pop.astype(dtype={'NAME': 'str', 'POP':'float','state':'str','county':'str'})
pop[['COUNTY','STATE']] = pop['NAME'].str.split(', ', expand=True)
pop['COUNTY'] = pop['COUNTY'].str.replace(' County', '')
pop['COUNTY'] = pop['COUNTY'].str.replace(' Parish', '')
pop = pop[['FIPS','COUNTY','STATE','POP']].reset_index(drop=True)
pop.drop(pop[pop['STATE'].str.contains('Puerto Rico')].index, inplace=True)
pop = pop.sort_values(by='FIPS')
print(pop)

       FIPS      COUNTY    STATE       POP
2278  01001     Autauga  Alabama   55869.0
2331  01003     Baldwin  Alabama  223234.0
2390  01005     Barbour  Alabama   24686.0
2391  01007        Bibb  Alabama   22394.0
2396  01009      Blount  Alabama   57826.0
...     ...         ...      ...       ...
1241  56037  Sweetwater  Wyoming   42343.0
1240  56039       Teton  Wyoming   23464.0
1250  56041       Uinta  Wyoming   20226.0
1251  56043    Washakie  Wyoming    7805.0
1255  56045      Weston  Wyoming    6927.0

[3142 rows x 4 columns]


In [32]:
#Start central frame with Vet count and Veteran Percentage of Total Population
totpop_withvet = pd.merge(pop,POPveterans2021, on=AGKEY, how='left',suffixes=('','_drop'))
totpop_withvet.drop([col for col in totpop_withvet.columns if 'drop' in col],axis=1,inplace=True)
totpop_withvet['VET_PERCENT'] = totpop_withvet['VETS']/totpop_withvet['POP']
totpop_withvet.to_csv('totpop_withvet.csv',index=False)

In [68]:
#Daily updated NYTimes COVID-19 Data 
covid = pd.read_csv('https://github.com/nytimes/covid-19-data/blob/master/us-counties.csv?raw=true',dtype={'fips':'str'})

#Formatting of NYTimes COVID-19 Data
covid[['cases','deaths']] = covid[['cases','deaths']].fillna(0).astype(int)
covid['date'] = pd.to_datetime(covid['date'])
covid = covid[['date','fips','county','state','cases','deaths']]
covid = covid.rename(columns={'date':'DATE','fips':'FIPS','county':'COUNTY','state':'STATE','cases':'CASES','deaths':'DEATHS'})

In [69]:
#Creation of 2 dataframes isolating the latest days-worth of COVID-19 data by county and the day prior
covid_today = covid[covid['DATE'] == covid['DATE'].max()].reset_index(drop=True)
covid_yesterday = covid[covid['DATE'] == covid['DATE'].max()- pd.to_timedelta(1, unit='D')].reset_index(drop=True)

#Creation of merged dataframe for latest days-worth of data and day prior
covid_today2 = covid_today.drop(['DATE'],axis=1)
covid_yesterday2 = covid_yesterday.drop(['DATE'],axis=1).rename(columns={'CASES':'YESTER_CASES','DEATHS':'YESTER_DEATHS'})
covid_lately = pd.merge(covid_today2, covid_yesterday2, how='left', on=AGKEY, suffixes=('','_drop'))
covid_lately.drop([col for col in covid_lately.columns if 'drop' in col],axis=1,inplace=True)

print(covid_today)
print(covid_lately)

           DATE   FIPS      COUNTY    STATE  CASES  DEATHS
0    2021-04-26  01001     Autauga  Alabama   6882     107
1    2021-04-26  01003     Baldwin  Alabama  20863     305
2    2021-04-26  01005     Barbour  Alabama   2296      56
3    2021-04-26  01007        Bibb  Alabama   2584      62
4    2021-04-26  01009      Blount  Alabama   6574     134
...         ...    ...         ...      ...    ...     ...
3242 2021-04-26  56037  Sweetwater  Wyoming   4179      37
3243 2021-04-26  56039       Teton  Wyoming   3724       9
3244 2021-04-26  56041       Uinta  Wyoming   2165      12
3245 2021-04-26  56043    Washakie  Wyoming    897      26
3246 2021-04-26  56045      Weston  Wyoming    648       5

[3247 rows x 6 columns]
       FIPS      COUNTY    STATE  CASES  DEATHS  YESTER_CASES  YESTER_DEATHS
0     01001     Autauga  Alabama   6882     107          6879            107
1     01003     Baldwin  Alabama  20863     305         20847            305
2     01005     Barbour  Alabama   2

In [49]:

#Add in columns for today and yesterday's cases as well as the veteran share by population percentage
covid_lately2 = covid_lately.drop(['DEATHS','YESTER_DEATHS'], axis=1)
VAPopCases = pd.merge(totpop_withvet,covid_lately2,on=AGKEY, how='left',suffixes=('','_drop'))
VAPopCases.drop([col for col in VAPopCases.columns if 'drop' in col],axis=1,inplace=True)
VAPopCases['VET_CASES'] = VAPopCases['CASES'] * VAPopCases['VET_PERCENT']
VAPopCases['VET_YESTER'] = VAPopCases['YESTER_CASES'] * VAPopCases['VET_PERCENT']

print(VAPopCases)

       FIPS      COUNTY    STATE       POP          VETS  VET_PERCENT  \
0     01001     Autauga  Alabama   55869.0   5703.189466     0.102081   
1     01003     Baldwin  Alabama  223234.0  19665.803702     0.088095   
2     01005     Barbour  Alabama   24686.0   1848.845078     0.074894   
3     01007        Bibb  Alabama   22394.0   1264.859815     0.056482   
4     01009      Blount  Alabama   57826.0   3703.698806     0.064049   
...     ...         ...      ...       ...           ...          ...   
3137  56037  Sweetwater  Wyoming   42343.0   3161.770380     0.074670   
3138  56039       Teton  Wyoming   23464.0   1093.840038     0.046618   
3139  56041       Uinta  Wyoming   20226.0   1218.901127     0.060264   
3140  56043    Washakie  Wyoming    7805.0    621.987939     0.079691   
3141  56045      Weston  Wyoming    6927.0    549.049003     0.079262   

        CASES  YESTER_CASES    VET_CASES   VET_YESTER  
0      6882.0        6879.0   702.524654   702.218410  
1     20863

In [52]:
CDCcovid = pd.read_csv('COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv', usecols=['case_month','res_state','res_county','state_fips_code','county_fips_code','age_group'], dtype={'state_fips_code':'str','county_fips_code':'str'}) \
    .dropna(subset=['res_state','res_county','age_group']) \
    .groupby(['res_state','res_county','age_group'],sort=False,as_index=False) \
    .agg({'state_fips_code':'first','county_fips_code':'first','case_month':'count'}) \
    .reset_index(drop=True)

CDCcovid['FIPS'] = CDCcovid.county_fips_code
CDCcovid = CDCcovid.drop(['state_fips_code','county_fips_code'],axis=1).rename(columns={'res_state':'STATE','res_county':'COUNTY','age_group':'AGEGROUP','case_month':'CASES'})
CDCcovid = CDCcovid.drop(CDCcovid[CDCcovid['AGEGROUP'] == 'Missing'].index).sort_values(by=['FIPS','STATE','COUNTY','AGEGROUP'])
CDCcovid = CDCcovid[['FIPS','COUNTY','STATE','AGEGROUP','CASES']].reset_index(drop=True)
CDCcovid['COUNTY'] = CDCcovid['COUNTY'].str.title()
CDCcovid['STATE'] = CDCcovid.apply(lambda x: stateNames(x['STATE']), axis=1)

CDCcovid.head()

,FIPS,COUNTY,STATE,AGEGROUP,CASES
0,01001,Autauga,Alabama,0 - 17 years,489
1,01001,Autauga,Alabama,18 to 49 years,2062
2,01001,Autauga,Alabama,50 to 64 years,858
3,01001,Autauga,Alabama,65+ years,638
4,01003,Baldwin,Alabama,0 - 17 years,1185


In [39]:
  #Processing the CDC data to obtain columns for cases by county by age group and total cases by county
Cases1 = CDCcovid[CDCcovid['AGEGROUP'] == '18 to 49 years'].rename(columns={'CASES':'AG1CASES'}).drop(['AGEGROUP'],axis=1).reset_index(drop=True)
Cases2 = CDCcovid[CDCcovid['AGEGROUP'] == '50 to 64 years'].rename(columns={'CASES':'AG2CASES'}).drop(['AGEGROUP'],axis=1).reset_index(drop=True)
Cases3 = CDCcovid[CDCcovid['AGEGROUP'] == '65+ years'].rename(columns={'CASES':'AG3CASES'}).drop(['AGEGROUP'],axis=1).reset_index(drop=True)

Cases3 = Cases3.merge(AG3,on=AGKEY,how='left',suffixes=('','_drop')).merge(AG4,on=AGKEY,how='left',suffixes=('','_drop'))
Cases3.drop([col for col in Cases3.columns if 'drop' in col],axis=1,inplace=True)
Cases3['AG3CASES'] = Cases3['AG3CASES'] * (Cases3['TOTPOP3'] / (Cases3['TOTPOP3'] +Cases3['TOTPOP4']))
Cases3 = Cases3.drop(['TOTPOP3','TOTPOP4'], axis=1)

Cases4 = CDCcovid[CDCcovid['AGEGROUP'] == '65+ years'].rename(columns={'CASES':'AG4CASES'}).drop(['AGEGROUP'],axis=1).reset_index(drop=True)

Cases4 = Cases4.merge(AG3,on=AGKEY,how='left',suffixes=('','_drop')).merge(AG4,on=AGKEY,how='left',suffixes=('','_drop'))
Cases4.drop([col for col in Cases4.columns if 'drop' in col],axis=1,inplace=True)
Cases4['AG4CASES'] = Cases4['AG4CASES'] * (Cases4['TOTPOP4'] / (Cases4['TOTPOP3'] +Cases4['TOTPOP4']))
Cases4 = Cases4.drop(['TOTPOP3','TOTPOP4'], axis=1)


CDC = Cases1.merge(Cases2,on=AGKEY,how='left',suffixes=('','_drop')).merge(Cases3,on=AGKEY,how='left',suffixes=('','_drop')).merge(Cases4,on=AGKEY,how='left',suffixes=('','_drop'))
CDC.drop([col for col in CDC.columns if 'drop' in col],axis=1,inplace=True)

CDC['TOTALCASES'] = CDC['AG1CASES']+CDC['AG2CASES']+CDC['AG3CASES']+CDC['AG4CASES']

#Processing of these values to create county-level age-group percentage of cases
CDC['AG1AR'] = CDC['AG1CASES']/CDC['TOTALCASES']
CDC['AG2AR'] = CDC['AG2CASES']/CDC['TOTALCASES']
CDC['AG3AR'] = CDC['AG3CASES']/CDC['TOTALCASES']
CDC['AG4AR'] = CDC['AG4CASES']/CDC['TOTALCASES']

CDC.to_csv('CDC.csv',index=False)
print(CDC)
CDC.tail()

       FIPS     COUNTY      STATE  AG1CASES  AG2CASES     AG3CASES  \
0     01001    Autauga    Alabama      2062       858   571.082922   
1     01003    Baldwin    Alabama      5356      2358  1953.121332   
2     01005    Barbour    Alabama       569       262   227.686484   
3     01007       Bibb    Alabama       809       305   189.012590   
4     01009     Blount    Alabama      1751       777   605.440725   
...     ...        ...        ...       ...       ...          ...   
1465  55133   Waukesha  Wisconsin     18232      9627  5306.940723   
1466  55135    Waupaca  Wisconsin      1950      1141   799.359175   
1467  55137   Waushara  Wisconsin       662       457   253.100490   
1468  55139  Winnebago  Wisconsin      8303      3420  1885.519614   
1469  55141       Wood  Wisconsin      2895      1419   937.781931   

        AG4CASES  TOTALCASES     AG1AR     AG2AR     AG3AR     AG4AR  
0      66.917078      3558.0  0.579539  0.241147  0.160507  0.018807  
1     204.878668 

,FIPS,COUNTY,STATE,AG1CASES,AG2CASES,AG3CASES,AG4CASES,TOTALCASES,AG1AR,AG2AR,AG3AR,AG4AR
1465,55133,Waukesha,Wisconsin,18232,9627,5306.940723,832.059277,33998.0,0.536267,0.283164,0.156096,0.024474
1466,55135,Waupaca,Wisconsin,1950,1141,799.359175,140.640825,4031.0,0.483751,0.283056,0.198303,0.034890
1467,55137,Waushara,Wisconsin,662,457,253.100490,31.899510,1404.0,0.471510,0.325499,0.180271,0.022720
1468,55139,Winnebago,Wisconsin,8303,3420,1885.519614,301.480386,13910.0,0.596909,0.245866,0.135551,0.021674
1469,55141,Wood,Wisconsin,2895,1419,937.781931,154.218069,5406.0,0.535516,0.262486,0.173471,0.028527


In [40]:
AGProportions = AG.merge(AGVet,on=AGKEY,how='left',suffixes=('','_drop'))
AGProportions.drop([col for col in AGProportions.columns if 'drop' in col],axis=1,inplace=True)

AGProportions['Vet1Perc'] = AGProportions['VETPOP1']/AGProportions['TOTPOP1']
AGProportions['Vet2Perc'] = AGProportions['VETPOP2']/AGProportions['TOTPOP2']
AGProportions['Vet3Perc'] = AGProportions['VETPOP3']/AGProportions['TOTPOP3']
AGProportions['Vet4Perc'] = AGProportions['VETPOP4']/AGProportions['TOTPOP4']
AGProportions = AGProportions[['FIPS','COUNTY','STATE','Vet1Perc','Vet2Perc','Vet3Perc','Vet4Perc']]
print(AGProportions)

AGProportions.to_csv('AGProportions.csv',index=False)

       FIPS      COUNTY    STATE  Vet1Perc  Vet2Perc  Vet3Perc  Vet4Perc
0     01001     Autauga  Alabama  0.063252  0.150004  0.228088  0.307386
1     01003     Baldwin  Alabama  0.038631  0.097853  0.213776  0.377724
2     01005     Barbour  Alabama  0.034258  0.111533  0.165553  0.206797
3     01007        Bibb  Alabama  0.021856  0.072273  0.167355  0.187908
4     01009      Blount  Alabama  0.032224  0.071485  0.168916  0.270372
...     ...         ...      ...       ...       ...       ...       ...
3137  56037  Sweetwater  Wyoming  0.043609  0.093236  0.262040  0.266601
3138  56039       Teton  Wyoming  0.011929  0.052133  0.180907  0.123024
3139  56041       Uinta  Wyoming  0.029325  0.076286  0.210099  0.158415
3140  56043    Washakie  Wyoming  0.040875  0.077480  0.214019  0.154838
3141  56045      Weston  Wyoming  0.047753  0.083288  0.205848  0.062772

[3142 rows x 7 columns]


In [41]:
AgeFactor = VAPopCases.drop(['POP','VETS','VET_PERCENT','YESTER_CASES','VET_CASES','VET_YESTER'], axis=1)
AgeFactor = AgeFactor.merge(CDC, on=AGKEY,how='left',suffixes=('','_drop')).drop(['AG1CASES','AG2CASES','AG3CASES','AG4CASES','TOTALCASES'],axis=1)
AgeFactor.drop([col for col in AgeFactor.columns if 'drop' in col],axis=1,inplace=True)

AgeFactor['AG1AR'].fillna(value=AgeFactor['AG1AR'].mean(), inplace=True)
AgeFactor['AG2AR'].fillna(value=AgeFactor['AG2AR'].mean(), inplace=True)
AgeFactor['AG3AR'].fillna(value=AgeFactor['AG3AR'].mean(), inplace=True)
AgeFactor['AG4AR'].fillna(value=AgeFactor['AG4AR'].mean(), inplace=True)

AGCases = AgeFactor.merge(AGProportions, on=AGKEY, how='left',suffixes=('','_drop'))
AGCases.drop([col for col in AGCases.columns if 'drop' in col],axis=1,inplace=True)

AGCases['TOTALCASES1'] = AGCases['CASES']*AGCases['AG1AR']
AGCases['TOTALCASES2'] = AGCases['CASES']*AGCases['AG2AR']
AGCases['TOTALCASES3'] = AGCases['CASES']*AGCases['AG3AR']
AGCases['TOTALCASES4'] = AGCases['CASES']*AGCases['AG4AR']
AGCases = AGCases.drop(['CASES','AG1AR','AG2AR','AG3AR','AG4AR'],axis=1)

AGCases['VETCASES1'] = AGCases['TOTALCASES1']*AGCases['Vet1Perc']
AGCases['VETCASES2'] = AGCases['TOTALCASES2']*AGCases['Vet2Perc']
AGCases['VETCASES3'] = AGCases['TOTALCASES3']*AGCases['Vet3Perc']
AGCases['VETCASES4'] = AGCases['TOTALCASES4']*AGCases['Vet4Perc']
AGCases = AGCases.drop(['Vet1Perc','Vet2Perc','Vet3Perc','Vet4Perc'],axis=1)

print(AGCases)

       FIPS      COUNTY    STATE   TOTALCASES1  TOTALCASES2  TOTALCASES3  \
0     01001     Autauga  Alabama   3988.387858  1659.571669  1104.607272   
1     01003     Baldwin  Alabama  11319.107374  4983.281402  4127.630709   
2     01005     Barbour  Alabama   1206.301016   555.449677   482.703756   
3     01007        Bibb  Alabama   1577.702642   594.807547   368.610214   
4     01009      Blount  Alabama   3594.963773  1595.252342  1243.025399   
...     ...         ...      ...           ...          ...          ...   
3137  56037  Sweetwater  Wyoming   2429.382875   992.017509   668.449392   
3138  56039       Teton  Wyoming   2164.877202   884.008902   595.670145   
3139  56041       Uinta  Wyoming   1258.581939   513.931062   346.301252   
3140  56043    Washakie  Wyoming    521.454041   212.931253   143.479087   
3141  56045      Weston  Wyoming    376.702585   153.823246   103.650444   

      TOTALCASES4   VETCASES1   VETCASES2   VETCASES3   VETCASES4  
0      129.433201  

In [42]:
VetHospital = AGCases.drop(['TOTALCASES1','TOTALCASES2','TOTALCASES3','TOTALCASES4'],axis=1)

VetHospital['LOWER_Hospitalizations'] = 0.006*VetHospital['VETCASES1'] + 0.0545*VetHospital['VETCASES2'] + 0.159*VetHospital['VETCASES3'] + 0.2625*VetHospital['VETCASES4']

VetHospital['UPPER_Hospitalizations'] = 0.051*VetHospital['VETCASES1'] + 0.148*VetHospital['VETCASES2'] + 0.2625*VetHospital['VETCASES3'] + 0.45*VetHospital['VETCASES4']

VetHospital = VetHospital.drop(['VETCASES1','VETCASES2','VETCASES3','VETCASES4'],axis=1)

print(VetHospital)

       FIPS      COUNTY    STATE  LOWER_Hospitalizations  \
0     01001     Autauga  Alabama               65.584551   
1     01003     Baldwin  Alabama              212.430733   
2     01005     Barbour  Alabama               19.128562   
3     01007        Bibb  Alabama               14.473366   
4     01009      Blount  Alabama               50.284898   
...     ...         ...      ...                     ...   
3137  56037  Sweetwater  Wyoming               39.765930   
3138  56039       Teton  Wyoming               22.366154   
3139  56041       Uinta  Wyoming               15.847176   
3140  56043    Washakie  Wyoming                6.687245   
3141  56045      Weston  Wyoming                4.426421   

      UPPER_Hospitalizations  
0                 133.749484  
1                 399.694004  
2                  37.050271  
3                  27.940043  
4                  95.027643  
...                      ...  
3137               75.766910  
3138               40.823081  


In [88]:
CountyVACOVID = VAPopCases.merge(VetHospital,on=AGKEY,how='left',suffixes=('','_drop'))
CountyVACOVID.drop([col for col in CountyVACOVID.columns if 'drop' in col],axis=1,inplace=True)
TodayDate = covid['DATE'].max()
CountyVACOVID['DATE'] = ""
CountyVACOVID.loc[0,'DATE'] = '%s-%s-%s' % (TodayDate.month, TodayDate.day, TodayDate.year)
CountyVACOVID.to_csv('CountyVACOVID.csv' ,index=False)
CountyVACOVID.head()

,FIPS,COUNTY,STATE,POP,VETS,VET_PERCENT,CASES,YESTER_CASES,VET_CASES,VET_YESTER,LOWER_Hospitalizations,UPPER_Hospitalizations,DATE
0,01001,Autauga,Alabama,55869.0,5703.189466,0.102081,6882.0,6879.0,702.524654,702.218410,65.584551,133.749484,4-26-2021
1,01003,Baldwin,Alabama,223234.0,19665.803702,0.088095,20863.0,20847.0,1837.926403,1836.516883,212.430733,399.694004,
2,01005,Barbour,Alabama,24686.0,1848.845078,0.074894,2296.0,2296.0,171.957721,171.957721,19.128562,37.050271,
3,01007,Bibb,Alabama,22394.0,1264.859815,0.056482,2584.0,2584.0,145.949708,145.949708,14.473366,27.940043,
4,01009,Blount,Alabama,57826.0,3703.698806,0.064049,6574.0,6571.0,421.058277,420.866130,50.284898,95.027643,


In [121]:
VAMC = pd.read_csv('VAMC.csv', usecols=['NAME','STATE','STATEFP','COUNTYFP','CountyName','VISN'], converters={'STATEFP': '{:0>2}'.format,'COUNTYFP': '{:0>3}'.format}) \
    .dropna(subset=['VISN']) \
    .astype(dtype={'NAME': 'str', 'VISN':'int','STATEFP':'str','COUNTYFP':'str'}) 
    
VAMC['STATE'] = VAMC.apply(lambda x: stateNames(x['STATE']), axis=1)
VAMC['FIPS'] = VAMC.STATEFP + VAMC.COUNTYFP
VAMC = VAMC[['VISN','NAME','FIPS','CountyName','STATE']].rename(columns={'CountyName':'COUNTY','NAME':'VAMC'}).sort_values(by=['VISN','VAMC'])
VAMC.to_csv('CleanVAMC.csv',index=False)
VAMC.head()

,VISN,VAMC,FIPS,COUNTY,STATE
1495,1,Brockton VA Medical Center,44009,Washington,Massachusetts
1496,1,Brockton VA Medical Center,44007,Providence,Massachusetts
1497,1,Brockton VA Medical Center,44005,Newport,Massachusetts
1498,1,Brockton VA Medical Center,44003,Kent,Massachusetts
1499,1,Brockton VA Medical Center,44001,Bristol,Massachusetts


In [107]:
UScovid = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv')

#Formatting of NYTimes COVID-19 Data - Country Level
UScovid[['cases','deaths']] = UScovid[['cases','deaths']].fillna(0).astype(int)
UScovid['date'] = pd.to_datetime(UScovid['date'])
UScovid = UScovid.rename(columns={'date':'DATE','cases':'CASES','deaths':'DEATHS'})

USCasesToday = UScovid.loc[ UScovid.DATE == UScovid.DATE.max(),'CASES'].values[0]
USCasesYesterday = UScovid.loc[ UScovid.DATE == UScovid.DATE.max() - pd.to_timedelta(1, unit='D'),'CASES'].values[0]
USNewCases = USCasesToday - USCasesYesterday

47430

In [99]:

#DELETE THIS#
FacilityCases

,FIPS,COUNTY,STATE,POP,VETS,VET_PERCENT,CASES,YESTER_CASES,VET_CASES,VET_YESTER,LOWER_Hospitalizations,UPPER_Hospitalizations,DATE
555,16009,Benewah,Idaho,9298.0,943.092983,0.101430,667.0,666.0,67.653584,67.552154,7.342884,13.764002,
561,16021,Boundary,Idaho,12245.0,1154.957978,0.094321,860.0,859.0,81.115873,81.021552,7.999436,15.256911,
578,16055,Kootenai,Idaho,165697.0,14523.477635,0.087651,17673.0,17603.0,1549.052911,1542.917354,177.081902,336.409185,
579,16057,Latah,Idaho,40108.0,2380.432927,0.059351,3080.0,3066.0,182.799776,181.968868,10.679805,22.551659,
585,16069,Nez Perce,Idaho,40408.0,3366.307751,0.083308,3569.0,3557.0,297.326083,296.326388,31.306886,60.245546,
2953,53001,Adams,Washington,19983.0,591.755382,0.029613,2078.0,2074.0,61.535690,61.417238,10.225011,19.057363,
2954,53003,Asotin,Washington,22582.0,2022.624679,0.089568,1384.0,1384.0,123.962118,123.962118,13.111644,24.971495,
2959,53013,Columbia,Washington,3985.0,376.805413,0.094556,119.0,118.0,11.252157,11.157601,0.920284,1.800601,
2961,53017,Douglas,Washington,43429.0,2456.349558,0.056560,3588.0,3588.0,202.937719,202.937719,13.865300,28.252880,
2962,53019,Ferry,Washington,7627.0,715.284567,0.093783,304.0,296.0,28.510097,27.759831,2.447172,4.715938,


In [128]:
#State level Pulls
WA_Cases = CountyVACOVID[CountyVACOVID.STATE == 'Washington']['CASES'].sum()
OR_Cases = CountyVACOVID[CountyVACOVID.STATE == 'Oregon']['CASES'].sum()
ID_Cases = CountyVACOVID[CountyVACOVID.STATE == 'Idaho']['CASES'].sum()
OH_Cases = CountyVACOVID[CountyVACOVID.STATE == 'Ohio']['CASES'].sum()
OH_VACases = CountyVACOVID[CountyVACOVID.STATE == 'Ohio']['VET_CASES'].sum()

#VISN level Pulls
VISN20List = VAMC[VAMC.VISN == 20]['FIPS']
VISN20Data = CountyVACOVID[CountyVACOVID['FIPS'].isin(VISN20List)]

VISN20Cases = VISN20Data['CASES'].sum()
VISN20_VACases = VISN20Data['VET_CASES'].sum()

#Facility level Pulls
MGFacilityList = VAMC[VAMC.VAMC == 'Mann-Grandstaff Department of Veterans Affairs Medical Center']['FIPS']
MGFacilityData = CountyVACOVID[CountyVACOVID['FIPS'].isin(MGFacilityList)]

PSFacilityList = VAMC[VAMC.VAMC == 'Seattle VA Medical Center']['FIPS']
PSFacilityData = CountyVACOVID[CountyVACOVID['FIPS'].isin(PSFacilityList)]

#Hard-coded Columbus while VAMC list is broken/missing Columbus
ColumbusFacilityList = ['39159','39097', '39129','39049','39045','39089','39041','39117']
#ColumbusFacilityList = VAMC[VAMC.VAMC == '']['FIPS']
ColumbusFacilityData = CountyVACOVID[CountyVACOVID['FIPS'].isin(ColumbusFacilityList)]

ClevelandFacilityList = VAMC[VAMC.VAMC == 'Louis Stokes Cleveland Department of Veterans Affairs Medical Center']['FIPS']
ClevelandFacilityData = CountyVACOVID[CountyVACOVID['FIPS'].isin(ClevelandFacilityList)]

MG_ECases = MGFacilityData['VET_CASES'].sum()
MG_HospLower = MGFacilityData['LOWER_Hospitalizations'].sum()
MG_HospUpper = MGFacilityData['UPPER_Hospitalizations'].sum()

PS_ECases = PSFacilityData['VET_CASES'].sum()
PS_HospLower = PSFacilityData['LOWER_Hospitalizations'].sum()
PS_HospUpper = PSFacilityData['UPPER_Hospitalizations'].sum()

Columbus_ECases = ColumbusFacilityData['VET_CASES'].sum()
Columbus_HospLower = ColumbusFacilityData['LOWER_Hospitalizations'].sum()
Columbus_HospUpper = ColumbusFacilityData['UPPER_Hospitalizations'].sum()

Cleveland_ECases = ClevelandFacilityData['VET_CASES'].sum()
Cleveland_HospLower = ClevelandFacilityData['LOWER_Hospitalizations'].sum()
Cleveland_HospUpper = ClevelandFacilityData['UPPER_Hospitalizations'].sum()

In [142]:

#ONLY Part of the Cycle of Additions by Date for Python File

VAChart = pd.read_csv('VAChart.csv', parse_dates=['DATE'])

In [143]:
VAChart_newrow = pd.DataFrame({ 'DATE': TodayDate,
                           'US Cases': USCasesToday,
                           'New US Cases': USNewCases,
                           'VISN20 Cases': VISN20Cases,
                           'WA Cases': WA_Cases,
                           'OR Cases': OR_Cases,
                           'ID Cases': ID_Cases,
                           'VISN20 VACases': VISN20_VACases,
                           'OH Cases': OH_Cases,
                           'OH VACases': OH_VACases,
                           'MG ECases': MG_ECases,
                           'MG HospLower': MG_HospLower,
                           'MG HospUpper': MG_HospUpper,
                           'PS ECases': PS_ECases,
                           'PS HospLower': PS_HospLower,
                           'PS HospUpper': PS_HospUpper,
                           'Columbus ECases': Columbus_ECases,
                           'Columbus HospLower': Columbus_HospLower,
                           'Columbus HospUpper': Columbus_HospUpper,
                           'Cleveland ECases': Cleveland_ECases,
                           'Cleveland HospLower': Cleveland_HospLower,
                           'Cleveland HospUpper': Cleveland_HospUpper}, index=[0])

VAChart = pd.concat([VAChart_newrow, VAChart]).reset_index(drop=True).drop_duplicates(subset='DATE',keep='first').round(2)

VAChart.to_csv('VAChart.csv',index=False)

In [144]:
VAMC.dtypes

VISN       int64
VAMC      object
FIPS      object
COUNTY    object
STATE     object
dtype: object

In [6]:
### Processing of the mega-file from CDC

#CDCcovid = pd.read_csv('data_folder/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv', usecols=['case_month','res_state','res_county','state_fips_code',#'county_fips_code','age_group'], dtype={'state_fips_code':'str','county_fips_code':'str'}) \
 #   .dropna(subset=['res_state','res_county','age_group']) \
 #   .groupby(['res_state','res_county','age_group'],sort=False,as_index=False) \
 #   .agg({'state_fips_code':'first','county_fips_code':'first','case_month':'count'}) \
 #   .reset_index(drop=True)

#CDCcovid['FIPS'] = CDCcovid.county_fips_code
#CDCcovid = CDCcovid.drop(['state_fips_code','county_fips_code'],axis=1).rename(columns={'res_state':'STATE','res_county':'COUNTY','age_group':'AGEGROUP',#'case_month':'CASES'})
#CDCcovid = CDCcovid.drop(CDCcovid[CDCcovid['AGEGROUP'] == 'Missing'].index).sort_values(by=['FIPS','STATE','COUNTY','AGEGROUP'])
#CDCcovid = CDCcovid[['FIPS','COUNTY','STATE','AGEGROUP','CASES']].reset_index(drop=True)
#CDCcovid['COUNTY'] = CDCcovid['COUNTY'].str.title()
#CDCcovid['STATE'] = CDCcovid.apply(lambda x: stateNames(x['STATE']), axis=1)

#CDCcovid.to_csv('data_folder/CDCcovid.csv', index=False)

